In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import time

rootPath = "/content/drive/MyDrive/dataset/veremi"
veremiPath = "/veremi"
t2Path = "/veremiWithT2"
csvt2Path = "/simulationscsv"

os.chdir(rootPath+t2Path+csvt2Path)

os.listdir()[-1:]

['veins_maat.uc1.14506012.180205_200240.csv']

In [ ]:
import json
os.chdir(rootPath+t2Path+csvt2Path)
concluded = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
startsim = concluded.last_valid_index() + 1
os.chdir(rootPath+veremiPath)
simulations = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
totalsim = len(simulations)
print('Total = ', totalsim, 'files')
print('Concluded = ', startsim - 1, 'files')
start_time = time.time()
mid = 'work/ul/ul_vertsys/ul_wqy57'
end = 'veins-maat/simulations/securecomm2018/results'
for idsim, sims in enumerate(simulations[startsim:totalsim]):
  # if idsim < 1:
    os.chdir(rootPath+veremiPath+'/'+sims+'/'+mid+'/'+sims+'/'+end)
    df = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
    df = df.drop(index=df.loc[df.str.contains('.json') == False].index).reset_index(drop=True)
    
    dataGT = []
    with open(df[0]) as f:
      for line in f:
        try:
          dataGT.append(json.loads(line.strip()))
        except ValueError:
          pass

    dataGT = pd.DataFrame(dataGT)
    # display(dataGT)

    dataLog = []
    lfiles = len(df[1:])
    for iditem, item in enumerate(df[1:], start=1):
      print('\r',
            str(idsim+1)+'º | ',
            'reading simulation '+str(startsim+idsim)+' of '+str(totalsim)+' |',
            " in {:.2f}".format(time.time() - start_time)+' sec',
            sep='', end='', flush=True)
      with open(df[iditem]) as f:
        for line in f:
          try:
            row = json.loads(line.strip())
            # add a receiver line to form the pair of communication
            row["receiver"] = int(df[iditem].split("-")[2])
            dataLog.append(row)
          except ValueError:
            pass

    dataLog = pd.DataFrame(dataLog)
    # add the last know receiver's position 
    vehicles = pd.Series(dataLog['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
    size = len(vehicles)
    # iv = vehicle index
    for iv, v in enumerate(vehicles):
      # if iv <= 1:
        # for each BSM (type 3) do this
        for tp in dataLog.loc[(dataLog['type'] == 3) & (dataLog['receiver'].astype(int) == v)].index:
          print('\r',
                str(idsim+1)+'º | ',
                'simulation '+str(startsim+idsim)+' of '+str(totalsim)+' |',
                ' vehicle '+str(iv+1)+' of '+str(size),
                " in {:.2f}".format(time.time() - start_time)+' sec',
                sep='', end='', flush=True)
          poslist = dataLog['pos'].iloc[dataLog.loc[(dataLog['receiver'].astype(int) == v) & (dataLog['type'] == 2)][0:tp].last_valid_index()]
          spdlist = dataLog['spd'].iloc[dataLog.loc[(dataLog['receiver'].astype(int) == v) & (dataLog['type'] == 2)][0:tp].last_valid_index()]
          dataLog.at[tp, 'pxRcv'] = poslist[0]
          dataLog.at[tp, 'pyRcv'] = poslist[1]
          dataLog.at[tp, 'pzRcv'] = poslist[2]
          dataLog.at[tp, 'sxRcv'] = spdlist[0]
          dataLog.at[tp, 'syRcv'] = spdlist[1]
          dataLog.at[tp, 'szRcv'] = spdlist[2]

    # Drop “Type2” in JSON files
    dataLog = dataLog.drop(dataLog[dataLog.type == 2].index).reset_index(drop=True)
    # Merge with ground truth to add labels
    dataLog = dataLog.merge(dataGT[['messageID', 'attackerType']])
    # display(dataLog)
    os.chdir(rootPath+t2Path+csvt2Path)
    print('\r',
          'saving',
          sep='', end='', flush=True)
    dataLog.to_csv(sims+'.csv')
    print('\r',
          sims+' done in {:.2f}'.format(time.time() - start_time)+' sec\n',
          sep='', end='', flush=True)

Total =  225 files
Concluded =  216 files
veins_maat.uc1.14506013.180205_200227 done in 10031.92 sec
veins_maat.uc1.14506014.180205_200240 done in 20479.88 sec
veins_maat.uc1.14506015.180205_200240 done in 30804.25 sec
veins_maat.uc1.14506016.180205_200240 done in 40938.96 sec
veins_maat.uc1.14506018.180205_200359 done in 50847.17 sec
veins_maat.uc1.14506020.180205_200359 done in 60000.92 sec
veins_maat.uc1.14506021.180205_200359 done in 69995.75 sec
veins_maat.uc1.14506023.180205_200518 done in 82266.48 sec
